### Commands

In [16]:
from firecloud import api as firecloud_api
import fc_interface

In [17]:
%load_ext autoreload
%autoreload 2

In [69]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

---

---

In [93]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [95]:
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [188]:
paths_to_batches_info = "paths_to_batches_info.xlsx"

In [189]:
# Retrieve list of samples with corresponding cohort from bsp.broadinstitute.org
samples_with_cohort = pd.read_excel('cohort_data/bsp_data.xls')

# All samples, without cohort data
all_samples = fc_interface.get_samples_multiple_batches(paths_to_batches_info, google_bucket_id)

# Add cohort data to all samples
data = pd.merge(all_samples, samples_with_cohort[['Sample ID', 'Collection']], \
                 left_on='bsp_sample_id_validation', \
                 right_on='Sample ID', \
                 how='outer') \
                .drop(['Sample ID'], axis=1)

# FC doesn't accept cohort names with non-alphanumeric characters, so use cohort codes instead
# Load dictionary of {long cohort name : short cohort code}
cohort_formatted_names = pd.read_table('cohort_files/cohort_names_dictionary.txt', \
                                       header=None, names=['cohort_name', 'cohort_code'])

# Add cohort codes to data
data = pd.merge(data, cohort_formatted_names, \
                        left_on='Collection', right_on='cohort_name', how='outer').\
                        drop(['cohort_name'], axis=1)

# Prepare for FC export format
data = data.rename(columns={'cohort_code': 'membership:sample_set_id', 'entity:sample_id': 'sample_id'})
# data = data[['membership:sample_set_id', 'sample_id']]

In [193]:
data.loc[ pd.isnull(data['membership:sample_set_id']), 'tsca_id'].unique()

array(['TSCA1213', 'TSCA15', 'TSCA20', 'TSCA16'], dtype=object)

In [187]:
samples_with_cohort.shape

(760, 70)

In [197]:
data[data['tsca_id'] == "TSCA16"]

,sample_id,participant_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code,bam_filename,bai_filename,clean_bai_file_capture,tsca_id,Collection,membership:sample_set_id
48,AB053-Tumor-SM-DNUMO,AB053,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AB053T_OPAC_p4,TSCA Rapid Cancer Detection Panel v2,SM-DOEKQ,SM-DNUMO,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AB053T_OPAC_p4,Primary,TP,2_AB053T_OPAC_p4_H5TTJBCXY.2.aligned.duplicate...,2_AB053T_OPAC_p4_H5TTJBCXY.2.aligned.duplicate...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Adam Bass/Ku...,AB
101,BT1011-Tumor-SM-DNUNI,BT1011,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT1011_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOELK,SM-DNUNI,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,BT1011_CM_p5,Primary,TP,2_BT1011_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,2_BT1011_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
102,BT1015-Tumor-SM-DNULF,BT1015,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT1015_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOEJH,SM-DNULF,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",BT1015_CM_p5,Primary,TP,2_BT1015_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,2_BT1015_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
103,BT1018-Tumor-SM-DNULG,BT1018,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT1018_NSA_Laminin_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOEJI,SM-DNULG,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,BT1018_NSA_Laminin_p5,Primary,TP,2_BT1018_NSA_Laminin_p5_H5TTJBCXY.2.aligned.du...,2_BT1018_NSA_Laminin_p5_H5TTJBCXY.2.aligned.du...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
104,BT1037-Tumor-SM-DNUNG,BT1037,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT1037_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOELI,SM-DNUNG,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",BT1037_CM_p5,Metastatic,TM,2_BT1037_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,2_BT1037_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
105,BT1038-Tumor-SM-DNUMR,BT1038,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT1038_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOEKT,SM-DNUMR,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",BT1038_CM_p5,Metastatic,TM,2_BT1038_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,2_BT1038_CM_p5_H5TTJBCXY.2.aligned.duplicates_...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
106,BT954-Tumor-SM-DO3CX,BT954,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT954_NSA_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOEJ7,SM-DO3CX,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",BT954_NSA_p5,Recurrent,TR,2_BT954_NSA_p5_H5TTJBCXY.2.aligned.duplicates_...,2_BT954_NSA_p5_H5TTJBCXY.2.aligned.duplicates_...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
107,BT970-Tumor-SM-DNUM8,BT970,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,BT970_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DOEKA,SM-DNUM8,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",BT970_CM_p5,Primary,TP,2_BT970_CM_p5_H5TTJBCXY.2.aligned.duplicates_m...,2_BT970_CM_p5_H5TTJBCXY.2.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA16,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,BT
108,BT990-Tumor-SM-DO3D1,BT990,gs://fc-35446f22-ea37-48